# News counter

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

import time
from datetime import datetime

### 급상승 검색어

In [ ]:
#
# 급상승 검색어
#

def get_keyword(cnt = 20):

    url = 'https://datalab.naver.com/keyword/realtimeList.naver'

    #header 추가
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' \
                      'AppleWebKit/537.36 (KHTML, like Gecko) ' \
                      'Chrome/58.0.3029.110 Safari/537.36',
    }
        
    req = requests.get(url,headers=header)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    lst = []
    for i, tag in enumerate(soup.select('span[class=item_title]')):
        #print("<{:03d}> {}\n".format(i+1, tag.text))
        
        lst.append(tag.text)
        
        if i >= cnt-1: break
    
    return lst


### News keyword 설정

In [ ]:
# keyword 정의
lst_kword1 = ['코로나', '선거']

In [ ]:
# keyword 파일에서 읽기
df = pd.read_excel('data/keyword.xlsx', header=None)
lst_kword2 =  df[0].values.tolist()

In [ ]:
# 급상승 검색에서 가져오기
lst_kword3 = get_keyword()

In [ ]:
lst_kword = lst_kword1 + lst_kword2 + lst_kword3

#lst_kword = lst_kword1

In [ ]:
cur = datetime.now().strftime('%Y-%m-%d')

# 사용자 설정: 검색일 설정(option)
#cur = '2019-04-28'

s_date = cur.replace('-','.')
e_date = cur.replace('-','.')

s_from = cur.replace('-','')
e_to   = cur.replace('-','')

### Main process

In [ ]:

f = open('data/naver_news_' + cur + '.csv', 'w')

f.write("키워드,날짜,Count\n")

for i, kword in enumerate(lst_kword):

    cnt = 0

    url = 'https://search.naver.com/search.naver?where=news&query=' + \
           kword + \
          '&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3' + \
          '&ds=' + s_date + \
          '&de=' + e_date + \
          '&docid=&nso=so%3Ar%2Cp%3A' + \
          'from' + s_from + \
          'to'   + e_to + \
          '%2Ca%3Aall&mynews=0&refresh_start=0&related=0'
          
    #header 추가
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    
    print(url)
    
    req = requests.get(url,headers=header)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    #print(soup)
    
    lst_title = soup.select('a.news_tit')
    if len(lst_title) == 0:
        print('검색 결과가 없습니다.')
        continue
    
    for j, item in enumerate(lst_title):
        print('<{:02d}> {}'.format(j, item.text))
    
    cnt = j + 1
    
    
    print('-' * 60)
    print("<{:03d}> keyword: {}, Date: {}, Count: {}".format(i, kword, cur, cnt))
    print('-' * 60)
    print('\n')
    
    f.write("{},{},{}\n".format(kword, cur, cnt))
    
    #
    # sleep 1 second
    #
    time.sleep(1)
                
f.close()

print('--------------------------------------------------------')
print('End process...')
print('--------------------------------------------------------')
    

In [ ]:
# end of file